<a href="https://colab.research.google.com/github/HuyenNguyenHelen/Huyen_INFO5731_Spring2020/blob/master/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#wuhancoronovirus"](https://twitter.com/hashtag/wuhancoronovirus) from Twitter. 


In [62]:
# Write your code here
import urllib.request
from bs4 import BeautifulSoup
url = "https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv"
request = urllib.request.urlopen (url)
webpage  = request.read()
soup = BeautifulSoup(webpage)

review = soup.find('select', {'class':'lister-list' })
#print(review)
title, content, date, rating = [], [], [], []

content_soup=soup.find_all("div", class_="text show-more__control")
title_soup = soup.find_all("a", class_="title")
date_soup = soup.find_all("span", class_="review-date")
rating_soup = soup.find_all("span", class_="rating-other-user-rating")
users_soup = soup.find_all("span", class_="display-name-link" )
for element in users_soup:
  user_links = element.next_element
  user_names= user_links.get_text()

for a,b,c,d in zip(content_soup,title_soup, date_soup, rating_soup):
  content.append(a.get_text())
  title.append(b.get_text())
  date.append(c.get_text())
  rating.append(d.getText())


print(content)
print(title)
print(date)
print(user_links)
print(user_names)
print(rating)

#i = item.next_element # links of users'prrofile
 # print(i.get_text())  # users'name
#print(user)
rating = soup.find_all("span", class_="rating-other-user-rating")

#for r in rating:
 # print(r.getText()) # rating

["I was a person that saw all the hype and claims of masterpiece as overreacting and overblown excitement for another Joker based film. I thought this looked solid at best and even a bit too pretentious in the trailer, but in here to say I was incredibly wrong. This is a massive achievement of cinema that's extremely rare in a day and age of cgi nonsense and reboots. While this is somewhat of a reboot of sorts, the standalone origin tale is impeccable from start to finish and echoes resemblance to the best joker origin comics from the past. Joaquin bleeds, sweats, and cries his every drop into this magnificently dedicated performance. Heath Ledger would be proud. This is undoubtedly the greatest acting performance since Heath's joker. The directing and writing is slickly brilliant and the bleak settings and tones are palpable throughout. When this film was over the place was blown away and every audience member was awestruck that they witnessed a film that could still transport them in

In [0]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [0]:
# Write your code here





# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [0]:
# Write your code here






**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 